In [1]:
## Download two clinical texts, with drug entites manually annotated
import requests
import os
import tarfile

#remote
url_root = "https://raw.githubusercontent.com/TeamHeka/data/main/"
archive="mtsamplesen.tar.gz"
url=url_root+archive

#local
local = os.getcwd()
path=local+"/data/"
if not os.path.isdir(path):
    print("Create dir "+path)
    # directory does not exists
    os.mkdir(path)

response = requests.get(url, stream=True)
file = tarfile.open(fileobj=response.raw, mode="r|gz")
file.extractall(path)

mypath=path+"mtsamplesen"
print("Data dir: "+mypath)

Download dir: mtsamplesen.tar.gz untared in /Users/coulet/workspace/medkit/docs/adrien/data/
Data dir: /Users/coulet/workspace/medkit/docs/adrien/data/mtsamplesen


In [39]:
## Read text doc as medkit documents
from pathlib import Path
from medkit.core.text import TextDocument

docs = TextDocument.from_dir(path=Path(mypath), pattern='[A-Z0-9].txt', encoding='utf-8')

In [4]:
## Create and run a three-step doc pipeline that: 
# 1) split sentences in texts 
# 2) Recognize drug entities with NER1: a dictionnary-based approach named UMLSMatcher 
# 3) Recognize drug entities with NER2: a Transformer-based approach, see https://huggingface.co/samrawal/bert-large-uncased_med-ner
from medkit.core import DocPipeline
from medkit.core import Pipeline
from medkit.core import PipelineStep
from medkit.text.segmentation import SentenceTokenizer
from medkit.text.ner import UMLSMatcher
from medkit.text.ner.hf_entity_matcher import HFEntityMatcher

# Def of the first step =============
# By default, SentenceTokenizer will use a list of punctuation chars to detect sentences.
sentence_tokenizer = SentenceTokenizer(
    # Label of the segments created and returned by the operation
    output_label="sentence",
    # Keep the punctuation character inside the sentence segments
    keep_punct=True,
    # Also split on newline chars, not just punctuation characters
    split_on_newlines=True,
)

# Def of the second step =============
# Codes of UMLS semantic groups to take into account
umls_semgroups = ["CHEM"]  # chemical (the UMLS semantic group to take into account)
umls_matcher = UMLSMatcher(
    # Directory containing the UMLS files with terms and concepts
    umls_dir="../data/UMLS/2023AB/META/",
    # Language to use (English)
    language="ENG",
    # Where to store the temp term database of the matcher
    cache_dir=".umls_cache/",
    # Semantic groups to consider
    semgroups=umls_semgroups,
    # Don't be case-sensitive
    lowercase=True,
    # Convert special chars to ASCII before matching
    normalize_unicode=True,
    name="umls_matcher"
)

# Def of the third step =============
bert_matcher = HFEntityMatcher(model="samrawal/bert-large-uncased_med-ner", name="bert_large")# an alternate model: "Clinical-AI-Apollo/Medical-NER"


pipeline3 = Pipeline(steps=[
    PipelineStep(sentence_tokenizer, input_keys=["full_text"], output_keys=["sentence"]),
    # deidentification step could come here
    PipelineStep(umls_matcher, input_keys=["sentence"], output_keys=["ner1_drug"]), 
    PipelineStep(bert_matcher, input_keys=["sentence"], output_keys=["ner2_drug"])],
                                          input_keys=["full_text"],
                                          output_keys=["sentence", "ner1_drug", "ner2_drug"])

doc_pipeline = DocPipeline(pipeline=pipeline3)
doc_pipeline.run(docs)

/Users/coulet/miniconda3/envs/medkit/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/coulet/miniconda3/envs/medkit/lib/python3.8/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [34]:
## Load medkit documents with their manual annotations made with Brat (our ground truth)
from medkit.io.brat import BratInputConverter

# Define an Input Converter to load brat text and annot in medkit docs
brat_converter = BratInputConverter()
ref_docs = brat_converter.load(dir_path=mypath)

In [35]:
## Compute some stats
print(f"Nb of docs: {len(docs)}")
    
i=0
for doc in docs:
    print ("Doc "+doc.uid+": ")
    # On annotations made by NER1 and NER2
    sentence_nb = len(doc.anns.get(label="sentence"))
    print(f"\t{sentence_nb} sentences,")
    ner1_drug_nb = len(doc.anns.get(label="chemical"))
    print(f"\t{ner1_drug_nb} drugs found with NER1,")  
    ner2_drug_nb = len(doc.anns.get(label="m"))
    print(f"\t{ner2_drug_nb} drugs found with NER2,")  
    
    # On the manual annotation (our ground truth)
    gt_nb = len(ref_docs[i].anns.get(label="Drug"))
    print(f"\t{gt_nb} drugs manually annotated.")  
    i=+1

Nb of docs: 2
Doc a0b8880e-2e5f-11ef-ae9f-be2893d1605a: 
	47 sentences,
	2 drugs found with NER1,
	6 drugs found with NER2,
	5 drugs manually annotated.
Doc a0b89a2e-2e5f-11ef-ae9f-be2893d1605a: 
	120 sentences,
	17 drugs found with NER1,
	19 drugs found with NER2,
	15 drugs manually annotated.


In [36]:
## Evaluate performance metrics of the NER1 and NER2 tools
from medkit.text.metrics.ner import SeqEvalEvaluator
import pandas as pd

def results_to_df(_results, _title):
    results_list = list(_results.items())
    arranged_results = {"Entities": ['P', 'R', 'F1'], 
                        "all": [round(results_list[i][1], 2) for i in [0, 1, 2]]}
    accuracy = round(results_list[4][1], 2)

    for i in range(5, len(results_list), 4):
        key = results_list[i][0][:-10]
        arranged_results[key] = [round(results_list[n][1], 2) for n in [i, i + 1, i + 2]]

    df = pd.DataFrame(arranged_results, index=[f"{_title} (acc={accuracy})", '', '']).T
    return df

predicted_entities1=[]
predicted_entities2=[]
dfs = []

for doc in docs:
    predicted_entities1.append(doc.anns.get(label="chemical"))
    predicted_entities2.append(doc.anns.get(label="m"))

# Annotations of NER1 are labelled as 'chemical', NER2 as 'm', but as 'Drug' in the ground truth
# The following dic enables remappings various labels of the same type of entites
remapping= {"chemical": "Drug", "m": "Drug"}
evaluator = SeqEvalEvaluator(return_metrics_by_label=True, average='weighted', labels_remapping=remapping) 
# eval of NER2
results1 = evaluator.compute(ref_docs, predicted_entities1)
dfs.append(results_to_df(_results=results1, _title="NER1"))
#print(results_to_df(_results=results1, _title="umls_matcher"))
# eval of NER2
results2 = evaluator.compute(ref_docs, predicted_entities2)
dfs.append(results_to_df(_results=results2, _title="NER2"))

print(pd.concat(dfs, axis=1))

         NER1 (acc=0.99)             NER2 (acc=1.0)            
Entities               P     R    F1              P     R    F1
all                 0.89  0.85  0.87           0.76  0.95  0.84
Drug                0.89  0.85  0.87           0.76  0.95  0.84


In [38]:
## Write annotations of tool NER1 in the brat format
from medkit.io.brat import BratOutputConverter

# reload raw documents
final_docs = TextDocument.from_dir(path=Path(mypath), pattern='[A-Z0-9].txt', encoding='utf-8')
# simplified pipeline, with only the best NER tool (NER1)
pipeline2 = Pipeline(steps=[
    PipelineStep(sentence_tokenizer, input_keys=["full_text"], output_keys=["sentence"]),
    # deidentification step could come here
    PipelineStep(umls_matcher, input_keys=["sentence"], output_keys=["ner1_drug"])],
                                          input_keys=["full_text"],
                                          output_keys=["ner1_drug"])

doc_pipeline2 = DocPipeline(pipeline=pipeline2)
doc_pipeline2.run(final_docs)
# Define Output Converter with default params,
# transfer all annotations and attributes
brat_output_converter = BratOutputConverter()

out_path="/Users/coulet/workspace/brat/brat-1.3p1/data/mtsamplesen/ner1_out/"

# save the annotation with the best tool (considering F1 only) in `out_path`
brat_output_converter.save(
  final_docs,  dir_path=out_path, doc_names=["doc_1", "doc_2"])